In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [76]:
dataset = pd.read_csv('C:\\Users\\sufya\\OneDrive\\Desktop\\Demand Forecasting\\final_dataset.csv')
dataset.head()

C:\Users\sufya\AppData\Local\Temp\ipykernel_18208\2653851893.py:1: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv('C:\\Users\\sufya\\OneDrive\\Desktop\\Demand Forecasting\\final_dataset.csv')


,order_item_id,product_id,customer_unique_id,Order ID,Order Date,CustomerName,City,Category,Sub-Category,Quantity
0,1.0,87285b34884572647811a353c7ac498a,7c396fd4830fd04220f754e42b4e5bff,e481f51cbdc54678b7cc49136f2d6af7,02-10-2017,NaN,sao paulo,housewares,NaN,1.0
1,1.0,87285b34884572647811a353c7ac498a,3a51803cc0d012c3b5dc8b7528cb05f7,128e10d95713541c87cd1a2e48201934,15-08-2017,NaN,sao paulo,housewares,NaN,1.0
2,1.0,87285b34884572647811a353c7ac498a,ef0996a1a279c26e7ecbd737be23d235,0e7e841ddf8f8f2de2bad69267ecfbcf,02-08-2017,NaN,sao paulo,housewares,NaN,1.0
3,1.0,87285b34884572647811a353c7ac498a,e781fdcc107d13d865fc7698711cc572,bfc39df4f36c3693ff3b63fcbea9e90a,23-10-2017,NaN,florianopolis,housewares,NaN,1.0
4,1.0,be021417a6acb56b9b50d3fd2714baa8,3e4fd73f1e86b135b9b121d6abbe9597,6ea2f835b4556291ffdc53fa0b3b95e8,24-11-2017,NaN,presidente venceslau,housewares,NaN,1.0


In [52]:
dataset.tail()

,order_item_id,product_id,customer_unique_id,Order ID,Order Date,CustomerName,City,Category,Sub-Category,Quantity
112518,NaN,Phones,NaN,B-26099,2019-03-30,Bhishm,Mumbai,Electronics,Phones,5.0
112519,NaN,Trousers,NaN,B-26099,2019-03-30,Bhishm,Mumbai,Clothing,Trousers,5.0
112520,NaN,Chairs,NaN,B-26100,2019-03-31,Hitika,Indore,Furniture,Chairs,2.0
112521,NaN,T-shirt,NaN,B-26100,2019-03-31,Hitika,Indore,Clothing,T-shirt,2.0
112522,NaN,Shirt,NaN,B-26100,2019-03-31,Hitika,Indore,Clothing,Shirt,2.0


In [77]:
# the number of null values in each column
print(dataset.isnull().sum())

order_item_id           1500
product_id              1500
customer_unique_id      1500
Order ID                   0
Order Date                 0
CustomerName          111023
City                       0
Category                   0
Sub-Category          111023
Quantity                   0
dtype: int64


In [73]:
# number of unique values in product_id column
print(dataset['product_id'].nunique())

32345


In [82]:
dataset.dtypes

order_item_id                float64
product_id                  category
customer_unique_id            object
Order ID                      object
Order Date            datetime64[ns]
CustomerName                  object
City                          object
Category                      object
Sub-Category                  object
Quantity                     float64
dtype: object

In [81]:
# categorize the product_id column
dataset['product_id'] = dataset['product_id'].astype('category')

In [80]:
dataset['Order Date'] = pd.to_datetime(dataset['Order Date'], dayfirst=False)

C:\Users\sufya\AppData\Local\Temp\ipykernel_18208\1058329609.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  dataset['Order Date'] = pd.to_datetime(dataset['Order Date'], dayfirst=False)


In [83]:
# find the range of dates
print('Min date: ', dataset['Order Date'].min())
print('Max date: ', dataset['Order Date'].max())

Min date:  2016-02-10 00:00:00
Max date:  2019-12-03 00:00:00


In [79]:
# where there is no value in product_id, add the value of sub-category from the same row
dataset['product_id'] = dataset['product_id'].fillna(dataset['Sub-Category'])


# MODEL

In [ ]:
# using the sales on the previous dates, now use xgbregressor to predict the sales on the next month of Decembe and also next year 2020
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# there is no column of sales in the dataset, each row represents a sale, and the quantity of the product sold is in the column 'Quantity'
# using this information, we have to predict the sales of the next month 
# we will use the 'Order Date' column to extract the month and year of the sale
# we will use the 'Quantity' column to get the sales of the product

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor, DMatrix
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

# Load the dataset
# dataset = pd.read_csv("sales_data.csv")  # Make sure to replace "sales_data.csv" with the actual filename

# Convert the Order Date column to datetime
dataset['Order Date'] = pd.to_datetime(dataset['Order Date'], format='%m-%d-%Y')

# Filter data for the next month
current_date = datetime.now()
next_month = current_date + timedelta(days=30)
next_month_data = dataset[dataset['Order Date'] >= current_date]
next_month_data = next_month_data[next_month_data['Order Date'] < next_month]

# Group the filtered data by product_id and sum the quantities
grouped_data = next_month_data.groupby('product_id')['Quantity'].sum().reset_index()

# Convert categorical 'product_id' to numerical labels
product_id_mapping = {product_id: i for i, product_id in enumerate(grouped_data['product_id'].unique())}
grouped_data['product_id'] = grouped_data['product_id'].map(product_id_mapping)

# Split data into train and test sets (since we are predicting for the next month, there is no need for splitting)
X_train = grouped_data[['product_id']]
y_train = grouped_data['Quantity']


In [88]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta

# Load the dataset
# data = pd.read_csv("sales_data.csv")  # Make sure to replace "sales_data.csv" with the actual filename

# Convert the Order Date column to datetime
dataset['Order Date'] = pd.to_datetime(dataset['Order Date'], format='%m-%d-%Y')

# Filter data for the next month
current_date = dataset['Order Date'].max()
next_month = current_date + timedelta(days=30)
# dataset does not have next month data, so we will use the last 3 months data
next_month_data = dataset[dataset['Order Date'] >= current_date - timedelta(days=90)]
next_month_data = next_month_data[next_month_data['Order Date'] < current_date]

# Group the filtered data by product_id and sum the quantities
grouped_data = next_month_data.groupby('product_id')['Quantity'].sum().reset_index()

# One-hot encode the 'product_id' column
grouped_data = pd.get_dummies(grouped_data, columns=['product_id'], prefix='product')

# Split data into train and test sets (since we are predicting for the next month, there is no need for splitting)
X_train = grouped_data.drop(columns=['Quantity'])
y_train = grouped_data['Quantity']

In [89]:
print(current_date)

2019-12-03 00:00:00


In [90]:
# Initialize and train XGBoost model
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [91]:
# Make predictions for the next month
next_month_product_ids = grouped_data.drop(columns=['Quantity']).columns  # Get the product_id columns from one-hot encoded dataframe
next_month_predictions = model.predict(grouped_data.drop(columns=['Quantity']))

# Display the predictions
predictions_df = pd.DataFrame({'product_id': next_month_product_ids, 'predicted_sales': next_month_predictions})
# sort the predictions in descending order
predictions_df = predictions_df.sort_values(by='predicted_sales', ascending=False)

predictions_df.head()


,product_id,predicted_sales
20204,product_Hankerchief,33.989937
20210,product_Shirt,30.990826
20212,product_Stole,22.990580
20207,product_Phones,16.990362
20200,product_Bookcases,14.989994


In [97]:
predictions_df

,product_id,predicted_sales
20204,product_Hankerchief,33.989937
20210,product_Shirt,30.990826
20212,product_Stole,22.990580
20207,product_Phones,16.990362
20200,product_Bookcases,14.989994
...,...,...
10777,product_54904bdd42a096ba337670dcc1b68a95,0.000003
10776,product_548e5bfe28edceab6b51fa707cc9556f,0.000003
10775,product_548ace38f22cc53db6f049c551d31397,0.000003
10774,product_5486b0f9faa264ae6cbbd2bf230bbb00,0.000003


In [96]:
# evaluate the model
actual_sales = y_train
predicted_sales = predictions_df['predicted_sales']

# Calculate the mean squared error
mse = mean_squared_error(actual_sales, predicted_sales)
print('Mean Squared Error:', mse)

# Calculate the root mean squared error
rmse = np.sqrt(mse)
# print('Root Mean Squared Error:', rmse)

# accuracy of the model
accuracy = (rmse / actual_sales.mean())
print('Accuracy:', accuracy)

Mean Squared Error: 0.23349321574354606
Accuracy: 83.13548589992324


In [ ]:
# now we will match the order of product(highest to lowest) with the actual sales of the product